In [1]:
"""
For this webscraper, the job description and requirements were combined into the same cell because they were not 
easily separated from the site's html. 
"""

import requests
from bs4 import BeautifulSoup as bs 
import re

import pandas as pd

url = 'https://r4d.csod.com/ats/careersite/search.aspx?site=2&c=r4d'

with requests.Session() as s: 
    r = s.get(url)
    src = r.content 
    soup = bs(src)
    soup.prettify()

    table2 = soup.find("table", class_ = 'results-table').find("tbody").find_all("tr")
    
    root_url = "https://r4d.csod.com/ats/careersite/"
    
    page_url = []     
    job_position = [] 
    location = []     
    des_and_req = []  
    Immunization = [] 
    Economics = []    
    
    allLinks = []
    allJobNames = []
    allLocations = []
    locationDataList = []
    
    for data in table2:
        job = data.find('a').text
        allJobNames.append(job)
        
        link = data.find('a')['href']
        complete_url = root_url + link
        allLinks.append(complete_url)
    
        locationData = data.find('span', class_ = "FieldValue").getText()
        locationDataList = locationData.split(" | ")
        locationString = locationDataList[2]
        locationString = locationString.replace("\xa0)","")
        moreString = ""
        if "More" in locationString:
            moreString = data.find('span', class_ = "FieldValue").find('a')['onclick']
            moreString = moreString.replace("showAllLocations('", "").replace("','", "\n").replace("','", "\n").replace("\\u003cbr/\\u003e", ", ")
        moreString = "\nMore - " + moreString
        locationString = locationString.replace("More", moreString)
        allLocations.append(locationString)
    
    
    index = 0
    for page in allLinks:
        result = s.get(page)
        page_source = result.content
        soup = bs(page_source) 
        soup.prettify()

        
        for script in soup(['script','style']):
            script.decompose()
        strips = list(soup.stripped_strings)
        strips = str(strips)

    
        immunization = ['Immunization', 'immunisation', 'vaccine', 'vaccines','vaccine-preventable diseases', 'vpd outbreak',
            'immunization campaign', 'SIA','supplemental immunization act ivities', 'cold chain', 'GAVI','shigella', 'cholera',
            'bcg', 'dtp', 'dpt', 'measles', 'influenza', 'conjugate vaccine']

        economics = ['Economics','expenditure tracking', 'financing', 
            'value for vaccination' , 'costing', 'economic analysis','costs' , 'equity', 'cost effectiveness', 'cost-effectiveness', 
            'cost benefit analysis', 'benefit-cost analysis','cost utility analysis','budget impact analysis' , 'budget' , 'budgeting' , 
            'GAVI','funding gap','fiscal']

        imm_result = any(ele in strips for ele in immunization)
        ec_result = any(ele in strips for ele in economics)
        
        if imm_result or ec_result:
            if imm_result: Immunization.append('True')
            else: Immunization.append('False')
            if ec_result: Economics.append('True')
            else: Economics.append('False')
            
            page_url.append(page) 
            job_position.append(allJobNames[index])
            location.append(allLocations[index])
        
            job_data = soup.find('div', class_='cs-atscs-jobdet-rtpane')

            desReqList= [desReqList.text.replace("\n"," ").replace("      ", " ") for desReqList in job_data.select('p, ul li')]
            desReqList.pop()
            popped = desReqList.pop(0)

            job_text = ""
            for elem in desReqList:
                job_text += elem + "\n"
                job_text = job_text.replace("\xa0"," ")
            des_and_req.append(job_text)
        index += 1
        
        
DataFrame = pd.DataFrame() 
DataFrame['Page Url']= page_url 
DataFrame['Job']= job_position 
DataFrame['Location'] = location
DataFrame['Description and Requirements'] = des_and_req
DataFrame['Immunization'] = Immunization
DataFrame['Economics'] = Economics

Data = DataFrame.drop_duplicates() 
Data.to_csv("R4D_Data.csv")

print('Webscraping complete')           

Webscraping complete
